<a href="https://colab.research.google.com/github/marcelo130102/Compiladores/blob/master/Compiladores_13042022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install flex
!apt-get install bison

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libfl-dev libfl2 libsigsegv2 m4
Suggested packages:
  bison flex-doc m4-doc
The following NEW packages will be installed:
  flex libfl-dev libfl2 libsigsegv2 m4
0 upgraded, 5 newly installed, 0 to remove and 40 not upgraded.
Need to get 545 kB of archives.
After this operation, 1,511 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 libsigsegv2 amd64 2.12-1 [14.7 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd64 m4 amd64 1.4.18-1 [197 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/main amd64 flex amd64 2.6.4-6 [316 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic/main amd64 libfl2 amd64 2.6.4-6 [11.4 kB]
Get:5 http://archive.ubuntu.com/ubuntu bionic/main amd64 libfl-dev amd64 2.6.4-6 [6,320 B]
Fetched 545 kB in 2s (293 kB/s)
Selecting previously unselected package libsigsegv

In [ ]:
%%writefile reconocedor.l
/* Programa que reconoce palabras */

/* Sección DEFINICIONES */
%{
#include <stdio.h>

%}

/* Sección REGLAS */
VERBO (es|estoy|soy|era|ser|hacer|hizo|puede|podria)
ADVERBIO (mucho|bastante|poco|muy|dentro|fuera|tampoco)
PREPOSICION (a|de|desde|durante|en|hasta|por|entre)
CONJUNCION (y|ni|que|pero|aunque|o|sino|siquiera|pues)
ADJETIVO (obediente|feliz|difícil|mío|tuyo|tres|cuatro|grande)
PRONOMBRE (yo|usted|él|ella|nosotros|ustedes|ellos|ellas)

%%
[ \t]+          /*ignorar espacios en blanco*/
{VERBO}         {printf("%s: es un verbo\n", yytext);}
{ADVERBIO}      {printf("%s: es un adverbio\n", yytext);}
{PREPOSICION}   {printf("%s: es una preposición\n", yytext);}
{CONJUNCION}    {printf("%s: es una conjunción\n", yytext);}
{ADJETIVO}      {printf("%s: es un adjetivo\n", yytext);}
{PRONOMBRE}     {printf("%s: es un pronombre\n", yytext);}
[a-zA-Z]+       {printf("%s: no se puede reconocer\n", yytext);}
.|\n            {ECHO;}
%%

/* Sección CODIGO USUARIO */
int yywrap() {}
int main(int argc, char **argv)
{
 	yylex();
	return 0;
}


Writing reconocedor.l


In [ ]:
%%writefile input.dat
yo estoy feliz
ese cuaderno es mío


Writing input.dat


In [ ]:
!flex reconocedor.l
!gcc lex.yy.c -o reco
!./reco < input.dat

yo: es un pronombre
estoy: es un verbo
feliz: es un adjetivo

ese: no se puede reconocer
cuaderno: no se puede reconocer
es: es un verbo
mío: es un adjetivo


In [ ]:
%%writefile  reco_tabla.l
/* Programa que reconoce palabras utilizando una 
tabla de simbolos */

/* Sección DEFINICIONES */
%{
#include <stdio.h>
enum{
    BUSCAR = 0,
    VERBO,
    ADJETIVO,
    ADVERBIO,
    SUSTANTIVO,
    PREPOSICION,
    PRONOMBRE,
    CONJUNCION
};

int estado;
int agregar_palabra(int tipo, char *palabra);
int buscar_palabra(char *palabra);
%}

/* Sección REGLAS */
%%
\n          {estado = BUSCAR;} /*retornar a estado default*/
^verbo      {estado = VERBO;}
^adjetivo   {estado = ADJETIVO;}
^adverbio   {estado = ADVERBIO;}
^sustantivo {estado = SUSTANTIVO;}
^preposicion {estado = PREPOSICION;}
^pronombre  {estado = PRONOMBRE;}
^conjuncion {estado = CONJUNCION;}

[a-zA-Z]+   {/*procesar palabras*/
                if (estado != BUSCAR) {
                    agregar_palabra(estado, yytext);
                }
                else {
                    switch ( buscar_palabra(yytext) ) {
                    case VERBO: printf("%s: verbo\n", yytext); break;
                    case ADJETIVO: printf("%s: adj.\n", yytext); break;
                    case ADVERBIO: printf("%s: adv.\n", yytext); break;
                    case SUSTANTIVO: printf("%s: sust.\n", yytext); break;
                    case PREPOSICION: printf("%s: prep.\n", yytext); break;
                    case PRONOMBRE: printf("%s: pron.\n", yytext); break;
                    case CONJUNCION: printf("%s: conj.\n", yytext); break;
                    default:
                        printf("%s: no se puede reconocer\n", yytext); 
                        break;
                    }
                }
            }/*fin procesar palabras*/
                
.           /*ignorar cualquier otro símbolo*/
%%

/* Sección CODIGO USUARIO */
int yywrap() {}
int main(int argc, char **argv)
{
 	yylex();
	return 0;
}

/* definir una lista enlazada de palabras y tipos*/
struct palabra {
    char *nombre_palabra;
    int tipo_palabra;
    struct palabra *sgte;
};

struct palabra *lista = NULL; /*primer elemento de la lista*/

int agregar_palabra(int tipo, char *palabra) {
    struct palabra *p; /*entrada de la lista*/
    
    if ( buscar_palabra(palabra) != BUSCAR ) {
        printf("La palabra %s ya fue definida!!!\n", palabra);
        return 0;
    }
    
    /* asignar espacio de memoria */
    p = malloc( sizeof( struct palabra ) );
    p->nombre_palabra = malloc( strlen(palabra) + 1 ); 
    /* copiar datos de palabra */
    strcpy( p->nombre_palabra, palabra );
    p->tipo_palabra = tipo;    
    /* enlazar nueva entrada a lista */
    p->sgte = lista;
    lista = p;
    return 1;
}

int buscar_palabra(char *palabra){
    struct palabra *p = lista;
    
    /* buscar palabra en la lista */
    while (p != NULL) {
        if ( strcmp(p->nombre_palabra, palabra) == 0 )
            return p->tipo_palabra;
        p = p->sgte;
    }
    
    return BUSCAR; /*no encontrado*/
}


Writing reco_tabla.l


In [ ]:
%%writefile input.dat
verbo es estoy soy era ser hacer hizo puede podria
es
sustantivo hombre gato planta mueble pelota
verbo comer correr salir
verbo dormir estudiar descansar
hombre correr
comer salir descansar
verbo hablar
mueble hablar soy 



Overwriting input.dat


In [ ]:
RESERVADA begin end while for case do if else 
TIPODATO int float double char vector array



In [ ]:
!flex reco_tabla.l
!gcc lex.yy.c -o reco
!./reco < input.dat

es: verbo
hombre: sust.
correr: verbo
comer: verbo
salir: verbo
descansar: verbo
mueble: sust.
hablar: verbo
soy: verbo


# Avance 25 - 04 - 2022

In [ ]:
%%writefile lexer.l
/* Programa (lexer) que será llamado desde un parser */

/* Sección DEFINICIONES */
%{
#include <stdio.h>
#include "parser.tab.h"

#define BUSCAR 0
int estado;
int agregar_palabra(int tipo, char *palabra);
int buscar_palabra(char *palabra);
%}

/* Sección REGLAS */
%%
\n          {estado = BUSCAR;}
^verbo      {estado = VERBO;}
^adjetivo   {estado = ADJETIVO;}
^adverbio   {estado = ADVERBIO;}
^sustantivo {estado = SUSTANTIVO;}
^preposicion {estado = PREPOSICION;}
^pronombre  {estado = PRONOMBRE;}
^conjuncion {estado = CONJUNCION;}

[a-zA-Z]+   {/*procesar palabras*/
                if (estado != BUSCAR) {
                    agregar_palabra(estado, yytext);
                }
                else {
                    switch ( buscar_palabra(yytext) ) {
                    case VERBO:{ printf("VERBO\n"); return VERBO;}
                    case ADJETIVO:{ printf("ADJETIVO\n"); return ADJETIVO;}
                    case ADVERBIO:{ printf("ADVERBIO\n"); return ADVERBIO;}
                    case SUSTANTIVO:{ printf("SUSTANTIVO\n"); return SUSTANTIVO;}
                    case PREPOSICION:{ printf("PREPOSICION\n"); return PREPOSICION;}
                    case PRONOMBRE:{ printf("PRONOMBRE\n"); return PRONOMBRE;}
                    case CONJUNCION:{ printf("CONJUNCION\n"); return CONJUNCION;}
                    default:
                        printf("%s: no se reconoce\n", yytext); 
                        /*no retornar nada*/
                    }
                }
            }/*fin procesar palabras*/
                
.           /*ignorar cualquier otro símbolo*/
%%

/* Sección CODIGO USUARIO */
/* definir una lista enlazada de palabras y tipos*/
struct palabra {
    char *nombre_palabra;
    int tipo_palabra;
    struct palabra *sgte;
};

struct palabra *lista = NULL; /*primer elemento de la lista*/

int agregar_palabra(int tipo, char *palabra) {
    struct palabra *p; /*entrada de la lista*/
    
    if ( buscar_palabra(palabra) != BUSCAR ) {
        printf("La palabra %s ya fue definida!!!\n", palabra);
        return 0; //no funciono
    }
    
    /* asignar espacio de memoria */
    p = malloc( sizeof( struct palabra ) );
    p->nombre_palabra = malloc( strlen(palabra) + 1 ); 
    /* copiar datos de palabra */
    strcpy( p->nombre_palabra, palabra );
    p->tipo_palabra = tipo;    
    /* enlazar nueva entrada a lista */
    p->sgte = lista;
    lista = p;
    return 1; //funciono
}

int buscar_palabra(char *palabra){
    struct palabra *p = lista;
    
    /* buscar palabra en la lista */
    while (p != NULL) {
        if ( strcmp(p->nombre_palabra, palabra) == 0 )
            return p->tipo_palabra;
        p = p->sgte;
    }
    
    return BUSCAR; /*no encontrado*/
}

Writing lexer.l


In [ ]:
%%writefile parser.y
/* Analizador sintactico para reconocer sentencias  */

/* Sección DEFINICIONES */
%{
#include <stdio.h>
//#define YYERROR_VERBOSE

int yylex();
int yyerror(char *s);
%}

/* Sección REGLAS */
%token SUSTANTIVO PRONOMBRE VERBO ADVERBIO ADJETIVO PREPOSICION CONJUNCION  

%%
sentencias: sentencia sentencias 
    | sentencia
    ;
sentencia:  sujeto VERBO objeto  {printf("La sentencia es valida: SVS\n");}
    ;
sujeto:     SUSTANTIVO
    |       PRONOMBRE
    ;
objeto:     SUSTANTIVO
    ;
%%

/* Sección CODIGO USUARIO */
FILE *yyin;
int main() {
    do {
        yyparse();
    }
    while ( !feof(yyin) );
    
    return 0;
}

int yyerror(char *s) {
    fprintf(stderr, "JS:%s\n", s);
    return 0;
}


Writing parser.y


In [ ]:
%%writefile input.dat
sustantivo hombre gato maiz mueble pelota
verbo come corre sale
hombre come pelota
verbo muerde vuela
gato muerde hola

Overwriting input.dat


In [ ]:
!flex lexer.l

In [ ]:
!bison -d parser.y

In [ ]:
!ls

input.dat  lex.yy.c  parser.tab.c  parser.y
lexer.l    parser.l  parser.tab.h  sample_data


In [ ]:
!gcc -c lex.yy.c parser.tab.c

In [ ]:
!ls

input.dat  lex.yy.c  parser.l	   parser.tab.h  parser.y
lexer.l    lex.yy.o  parser.tab.c  parser.tab.o  sample_data


In [ ]:
!gcc -o lexpar lex.yy.o parser.tab.o -ll

In [ ]:
!./lexpar < input.dat

SUSTANTIVO
VERBO
SUSTANTIVO
La sentencia es valida: SVS
SUSTANTIVO
VERBO
hola: no se reconoce
JS:syntax error
